# Volatility Surface Construction – Notebook 2
## Option Pricing Foundations & Implied Volatility

### 2.1 Motivation: From Prices to Volatility

In Notebook 1, we established a clean dataset of market option prices. However, raw prices are poor candidates for direct modeling. A call option price ($C$) depends heavily on the underlying spot price ($S$), the strike price ($K$), time to maturity ($T$), and interest rates ($r$). 

Because prices scale with the spot and decay with time, they are not comparable across different maturities or strikes. To construct a coherent surface, we must transform these heterogeneous prices into a normalized, dimensionless quantity: **Implied Volatility** ($\sigma_{imp}$).

Implied volatility represents the market's consensus estimate of the underlying asset's future variability. It serves as the "lingua franca" of options trading—a standardized metric that allows us to compare a deep out-of-the-money put expiring in 2 years with an at-the-money call expiring in 1 week.

In [1]:
import numpy as np
import sys
from pathlib import Path

# Ensure local modules can be imported
sys.path.append(str(Path.cwd().parent))

from src.discounting import discount_factor, FlatYieldCurve
from src.black_scholes import bs_price
from src.implied_vol import implied_volatility

### 2.2 Discounting and Forward Measure

Before pricing, we must normalize the time-value of money. This functionality is handled by `src.discounting`.

The **Discount Factor** $D(0, T)$ represents the present value of $1 received at time $T$. For a flat risk-free rate $r$:
$$ D(0, T) = e^{-rT} $$

More importantly, we operate in the **Forward Measure**. The Forward Price $F_0(T)$ is the theoretically arbitrage-free price of the underlying asset at maturity, adjusted for rates and dividends ($q$):

$$ F_0(T) = S_0 e^{(r-q)T} = \frac{S_0 e^{-qT}}{D(0, T)} $$

Using the forward price simplifies the Black-Scholes formula and allows us to define **Log-Moneyness** ($k$), which aligns the volatility smile around the ATM point ($k=0$):

$$ k = \ln\left(\frac{K}{F_0(T)}\right) $$

In [2]:
# Demonstration of discounting module
r = 0.05  # 5% risk-free rate
T = 1.0   # 1 year to maturity

# Calculate discount factor using the source script
df = discount_factor(rate=r, maturity=T, compounding="continuous")
print(f"Discount Factor D(0, 1.0): {df:.4f}")

Discount Factor D(0, 1.0): 0.9512


### 2.3 The Black-Scholes Pricing Framework

We utilize the Black-Scholes-Merton model not because we assume market dynamics are strictly log-normal (they are not), but because it serves as a standard **pricing operator** or transformation function. This logic resides in `src.black_scholes`.

The model maps our inputs to a theoretical price:
$$ C_{BS} = C(S, K, T, r, q, \sigma) $$

Where $\sigma$ is the volatility parameter. The standard formula for a Call option is:

$$ C(S, t) = S e^{-qT} N(d_1) - K e^{-rT} N(d_2) $$

By keeping the implementation of this formula in `src/black_scholes.py`, we ensure numerical stability and separate the deterministic math from our analysis.

In [3]:
# Demonstration of Black-Scholes pricing operator
S = 100.0
K = 100.0
sigma = 0.20  # 20% Volatility

price = bs_price(S, K, T, r, sigma, option_type="call")
print(f"Black-Scholes Call Price: ${price:.4f}")

Black-Scholes Call Price: $10.4506


### 2.4 Implied Volatility: The Inversion Problem

The market provides the Option Price ($C_{mkt}$), not the volatility. The **Implied Volatility** is the value $\sigma_{imp}$ that solves:

$$ C_{BS}(\sigma_{imp}) - C_{mkt} = 0 $$

**Key Properties:**
1.  **No Closed Form:** There is no analytic algebraic solution for $\sigma_{imp}$ in terms of $C_{mkt}$; we must find the root numerically.
2.  **Monotonicity:** The option price is strictly increasing with respect to volatility (positive Vega, $\nu > 0$). This guarantees that if a solution exists, it is unique.
3.  **The "Smile":** If the Black-Scholes assumptions held perfectly, $\sigma_{imp}$ would be constant across all strikes ($K$). In reality, we observe a "smile" or "skew," indicating the market prices in higher tail risks than the normal distribution predicts.

### 2.5 Numerical Implementation

The `src.implied_vol` module handles the numerical inversion. It employs **Brent's Method**, a robust root-finding algorithm that combines bisection, secant, and inverse quadratic interpolation. This is preferred over simple Newton-Raphson because it is guaranteed to converge as long as the root is bracketed.

We also handle edge cases:
* **Arbitrage Violations:** If $C_{mkt}$ is below the intrinsic value, no real $\sigma$ exists.
* **Deep ITM/OTM:** These options have very low Vega, making the inversion sensitive to price noise.

In [4]:
# Demonstration of Implied Volatility Inversion
market_price = 10.45  # Hypothetical observed price

iv = implied_volatility(
    price=market_price,
    S=S,
    K=K,
    T=T,
    r=r,
    option_type="call"
)

print(f"Market Price: ${market_price} -> Implied Volatility: {iv:.2%}")

Market Price: $10.45 -> Implied Volatility: 20.00%


### 2.6 The Transformation: Bridge to Surface Modeling

From this point forward, we abandon Option Prices ($C$) as our primary unit of analysis. We transform the filtered dataset from Notebook 1 into a sparse cloud of volatility points.

**The New Coordinates:**
1.  **X-Axis:** Log-Moneyness ($k = \ln(K/F)$)
2.  **Y-Axis:** Time to Maturity ($T$)
3.  **Z-Axis:** Implied Volatility ($\sigma_{imp}$)

This transformation reveals the structural features of the market—specifically the **Volatility Surface**—which we will model using the parametric SVI and SABR models in, respectively, **Notebook 3 and 4**.